<a href="https://colab.research.google.com/github/codewonka/ml-repository/blob/main/TensorFlow_Pockemon_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [TensorFlow-Pokemon-Course](https://lab.github.com/everydeveloper/advance-tensorflow) Learning Lab in GitHub.
- Predict Pokemon stats with the help of the TensorFlow library
- 목표 : 전설의 포켓몬인지(isLegendary) 여부 판단


In [ ]:
!nvidia-smi

Mon Jul  5 08:01:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Practice/TensorFlow-Pokemon-Course"

/content/drive/MyDrive/Colab Notebooks/Practice/TensorFlow-Pokemon-Course


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Kaggle Datasets 사용하기
### ( Datasets 이미 세팅했다면 PASS ) 
- Kaggle API Token 다운로드 및 Google Drive 에 업로드 한다.
- Kaggle Datasets를 API Command 를 활용하여 다운로드 한다.
- 다운로드 받은 Kaggle Datasets 를 Google Drive 에 업로드 한다.
- 업로드한 .zip datasets의 압축을 해제한다.

In [ ]:
# 캐글 설치 및 다운받은 API Token Colab으로 upload
# API Token은 kaggle > account > API 에서 다운로드 한다.
!pip install -U kaggle
from google.colab import files
files.upload()

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"codewonka","key":"fb41d6125dde32f09b32143eb60cb140"}'}

In [ ]:
# API Copy
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Datasets 다운받고 압축해제하기
!kaggle datasets download -d alopez247/pokemon
!unzip -qq "./pokemon.zip" -d "./datasets"

  0% 0.00/715k [00:00<?, ?B/s]
100% 715k/715k [00:00<00:00, 46.6MB/s]


In [ ]:
!ls -trl ./datasets/*.csv

-rw------- 1 root root 79392 Sep 20  2019 ./datasets/pokemon_alopez247.csv


# Pokemon Datasets 전처리

In [ ]:
df = pd.read_csv('./datasets/pokemon_alopez247.csv')
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [ ]:
# 연관성 있는 데이터의 범주로 좁히기
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]
df.columns

Index(['isLegendary', 'Generation', 'Type_1', 'Type_2', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Color', 'Egg_Group_1',
       'Height_m', 'Weight_kg', 'Body_Style'],
      dtype='object')

In [ ]:
# Data 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   isLegendary  721 non-null    bool   
 1   Generation   721 non-null    int64  
 2   Type_1       721 non-null    object 
 3   Type_2       350 non-null    object 
 4   HP           721 non-null    int64  
 5   Attack       721 non-null    int64  
 6   Defense      721 non-null    int64  
 7   Sp_Atk       721 non-null    int64  
 8   Sp_Def       721 non-null    int64  
 9   Speed        721 non-null    int64  
 10  Color        721 non-null    object 
 11  Egg_Group_1  721 non-null    object 
 12  Height_m     721 non-null    float64
 13  Weight_kg    721 non-null    float64
 14  Body_Style   721 non-null    object 
dtypes: bool(1), float64(2), int64(7), object(5)
memory usage: 79.7+ KB


In [ ]:
df.head()

,isLegendary,Generation,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Color,Egg_Group_1,Height_m,Weight_kg,Body_Style
0,False,1,Grass,Poison,45,49,49,65,65,45,Green,Monster,0.71,6.9,quadruped
1,False,1,Grass,Poison,60,62,63,80,80,60,Green,Monster,0.99,13.0,quadruped
2,False,1,Grass,Poison,80,82,83,100,100,80,Green,Monster,2.01,100.0,quadruped
3,False,1,Fire,NaN,39,52,43,60,50,65,Red,Monster,0.61,8.5,bipedal_tailed
4,False,1,Fire,NaN,58,64,58,80,65,80,Red,Monster,1.09,19.0,bipedal_tailed


- Model 이 읽을 수 있는 datasets 로 변환
  1. 숫자 타입으로 변환한다. ex) bool 타입 (True, False => 0, 1)
  2. 숫자 타입 변환이 적절하지 않은 경우, 더미 컬럼을 추가한다. ex) Object 타입 (물, 불)
    - 컬럼내의 값들을 뽑아내어 모두 다 신규 컬럼으로 생성하고, 0, 1 로 표현하도록 한다.

In [ ]:
# bool 타입 isLegendary 컬럼을 0, 1 로 변환
df['isLegendary'] = df['isLegendary'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   isLegendary  721 non-null    int64  
 1   Generation   721 non-null    int64  
 2   Type_1       721 non-null    object 
 3   Type_2       350 non-null    object 
 4   HP           721 non-null    int64  
 5   Attack       721 non-null    int64  
 6   Defense      721 non-null    int64  
 7   Sp_Atk       721 non-null    int64  
 8   Sp_Def       721 non-null    int64  
 9   Speed        721 non-null    int64  
 10  Color        721 non-null    object 
 11  Egg_Group_1  721 non-null    object 
 12  Height_m     721 non-null    float64
 13  Weight_kg    721 non-null    float64
 14  Body_Style   721 non-null    object 
dtypes: float64(2), int64(8), object(5)
memory usage: 84.6+ KB


In [ ]:
df.head()

,isLegendary,Generation,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Color,Egg_Group_1,Height_m,Weight_kg,Body_Style
0,0,1,Grass,Poison,45,49,49,65,65,45,Green,Monster,0.71,6.9,quadruped
1,0,1,Grass,Poison,60,62,63,80,80,60,Green,Monster,0.99,13.0,quadruped
2,0,1,Grass,Poison,80,82,83,100,100,80,Green,Monster,2.01,100.0,quadruped
3,0,1,Fire,NaN,39,52,43,60,50,65,Red,Monster,0.61,8.5,bipedal_tailed
4,0,1,Fire,NaN,58,64,58,80,65,80,Red,Monster,1.09,19.0,bipedal_tailed


In [ ]:
# dummy 컬럼을 추가하여 Object 타입 data를 옮기고, 기존컬럼을 삭제하는 함수 생성
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])   # pd.get_dummies : 데이터를 읽어서 더미 테이블 생성
        df = pd.concat([df, df_dummy], axis=1)   # pd.concat : 더미 테이블을 오리지날 테이블에 갖다 붙인다.
        df = df.drop(i, axis=1)  # 기존 컬럼을 삭제  ( axis = 1 은 열을 따라, axis = 0 은 행을 따라 가라는 의미 )
    return(df)

In [ ]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])   # Ojbect 컬럼들을 배열형태로 두번째 인자로 넣어준다

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 85 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   isLegendary       721 non-null    int64  
 1   Generation        721 non-null    int64  
 2   HP                721 non-null    int64  
 3   Attack            721 non-null    int64  
 4   Defense           721 non-null    int64  
 5   Sp_Atk            721 non-null    int64  
 6   Sp_Def            721 non-null    int64  
 7   Speed             721 non-null    int64  
 8   Height_m          721 non-null    float64
 9   Weight_kg         721 non-null    float64
 10  Amorphous         721 non-null    uint8  
 11  Bug               721 non-null    uint8  
 12  Ditto             721 non-null    uint8  
 13  Dragon            721 non-null    uint8  
 14  Fairy             721 non-null    uint8  
 15  Field             721 non-null    uint8  
 16  Flying            721 non-null    uint8  
 1

In [ ]:
df.head()

,isLegendary,Generation,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Height_m,Weight_kg,Amorphous,Bug,Ditto,Dragon,Fairy,Field,Flying,Grass,Human-Like,Mineral,Monster,Undiscovered,Water_1,Water_2,Water_3,bipedal_tailed,bipedal_tailless,four_wings,head_arms,head_base,head_legs,head_only,insectoid,multiple_bodies,quadruped,serpentine_body,several_limbs,two_wings,with_fins,Black,...,Purple,Red,White,Yellow,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,1,45,49,49,65,65,45,0.71,6.9,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,1,60,62,63,80,80,60,0.99,13.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,1,80,82,83,100,100,80,2.01,100.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,1,39,52,43,60,50,65,0.61,8.5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,58,64,58,80,65,80,1.09,19.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- 여기까지 Model 에서 사용가능한 데이터가 되었다.
- 지금부터 train, test 데이터셋으로 분리한다.


In [ ]:
# Generation 값을 기준으로 train, test 데이터를 분리하고, Generation 컬럼은 날린다.
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

In [ ]:
df_train, df_test = train_test_splitter(df, 'Generation')

In [ ]:
df_train.shape, df_test.shape

((570, 84), (151, 84))

In [ ]:
# train, test datasets 각각에서 Label 컬럼(isLegendary)을 분류한다
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [ ]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [ ]:
# isLegendary 컬럼을 라벨로 분류하고, 해당컬럼 삭제하여 data의 컬럼갯수가 줄어듬 (84 -> 83)
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

((570, 83), (570,), (151, 83), (151,))

# Keras 를 활용해 Model 만들기
- 두 개의 fully connected neural layer를 만든다.
1. 첫번째 Layer는 ReLU' (Rectified Linear Unit)
  - 첫 번째 층이기에 input size 를 특정해준다.
2. 마무리는 Softmax 층으로 한다.
  - 전설의 포켓몬이다. 아니다. 두 가지로 분류한다.

In [ ]:
# train_data 의 갯수를 length 로 지정
length = train_data.shape[1]
length

83

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

# Compile and Evaluate Model
 - compile
 - fit

In [ ]:
# optimizer : 모델이 정보를 얻으면서 어떻게 업데이트되는지를 결정하는 옵티마이저
# loss_function : 모델이 훈련될 때 얼마나 정확한지 측정하는 손실 함수
#                 one-hot encording 이 된다면 "categorial_crossentropy" 사용이 적절하다.
# metrics : 모델을 분석할 수 있도록 어떤 정보를 제공하는지 지정하는 메트릭을 선택
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=500)

Epoch 1/500
18/18 [==============================] - 0s 2ms/step - loss: 7.0812 - accuracy: 0.8930
Epoch 2/500
18/18 [==============================] - 0s 2ms/step - loss: 0.3316 - accuracy: 0.9246
Epoch 3/500
18/18 [==============================] - 0s 2ms/step - loss: 0.5000 - accuracy: 0.8912
Epoch 4/500
18/18 [==============================] - 0s 2ms/step - loss: 0.2179 - accuracy: 0.9333
Epoch 5/500
18/18 [==============================] - 0s 2ms/step - loss: 0.2085 - accuracy: 0.9333
Epoch 6/500
18/18 [==============================] - 0s 2ms/step - loss: 0.2114 - accuracy: 0.9404
Epoch 7/500
18/18 [==============================] - 0s 2ms/step - loss: 0.1888 - accuracy: 0.9456
Epoch 8/500
18/18 [==============================] - 0s 2ms/step - loss: 0.1833 - accuracy: 0.9439
Epoch 9/500
18/18 [==============================] - 0s 2ms/step - loss: 0.1780 - accuracy: 0.9404
Epoch 10/500
18/18 [==============================] - 0s 2ms/step - loss: 0.1798 - accuracy: 0.9351
Epoch 11/

# Predict legendary status using our model

In [ ]:
# test data를 통해 model 의 정확도를 테스트한다.
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print('Our test accuracy was "{0}"'.format(accuracy_value))

5/5 [==============================] - 0s 2ms/step - loss: 0.1425 - accuracy: 0.9536
Our test accuracy was "0.9536423683166504"


In [ ]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print('This was correctly predicted to be a "{0}"!'.format(test_labels[index]))
    else:
        print('This was incorrectly predicted to be a "{0}", It was actually a "{1}".'.format(np.argmax(prediction[index]), test_labels[index]))
        return(prediction)

# WHY? predict(test_data) 못맞추는 것일까??

In [ ]:
# 전설 포켓몬이 1 인데. predict 결과 1 나온 대상 중 실제(label) 1 은 없다.

prediction = model.predict(test_data)

for index in range(0, len(test_data)):
  if np.argmax(prediction[index]) == 1:
    print('np.argmax(prediction[{0}]) : '.format(index) + str(np.argmax(prediction[index])))
    print('test_labels[{0}]           : '.format(index) + str(test_labels[index]))

np.argmax(prediction[58]) : 1
test_labels[58]           : 0
np.argmax(prediction[94]) : 1
test_labels[94]           : 0
np.argmax(prediction[129]) : 1
test_labels[129]           : 0


In [ ]:
# predict 결과와 label이 안 맞는 것이 상당하다.
prediction = model.predict(test_data)

for index in range(0, len(test_data)):
  if np.argmax(prediction[index]) != train_labels[index]:
    print('np.argmax(prediction[{0}]) : '.format(index) + str(np.argmax(prediction[index])))
    print('test_labels[{0}]           : '.format(index) + str(test_labels[index]))

np.argmax(prediction[58]) : 1
test_labels[58]           : 0
np.argmax(prediction[91]) : 0
test_labels[91]           : 0
np.argmax(prediction[92]) : 0
test_labels[92]           : 0
np.argmax(prediction[93]) : 0
test_labels[93]           : 0
np.argmax(prediction[94]) : 1
test_labels[94]           : 0
np.argmax(prediction[97]) : 0
test_labels[97]           : 0
np.argmax(prediction[98]) : 0
test_labels[98]           : 0
np.argmax(prediction[129]) : 1
test_labels[129]           : 0
